<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C10-Windows_Privilege_Escalation/Windows_Token_Priviliges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Windows Token Privileges__*

A Windows __token__ is, per Microsoft, "an object that describes the security context of a process or thread" (see "Access Tokens" at https://msdn.microsoft). In other words, the token's permissions and privileges determine which tasks a process or thread can perform.

Misunderstanding these tokens can land you in trouble. As part of a security product, a well-intentioned developer might create a system tray application on which they'd like to give an unprivileged user the ability to control the main Windows service, which is a driver. The developer uses the native Windows API function __AdjustTokenPrivileges__ on the process and then, innocently enough, grants the system tray application the __SeLoadDriver__ privilege. What the developer doesn't notice is that if you can climb inside that system tray application, you now have the ability to load or unload any driver you want, which means you can drop a kernel mode rootkit, and that means game over.

Bear in mind that if you can't run your process monitor as SYSTEM or Administrator, then you need to keep an eye on what processes you __are__ able to monitor. Are there any additional privileges you can leverage? A process running as a user with the wrong privileges is a fantastic way to get to SYSTEM or run code in the kernel. __Table 10-1__ lists interesting privileges that the authors always look out for. It isn't exhaustive, but it serves as a good starting point. You can find a full list of privileges on the MSDN website.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C10-Windows_Privilege_Escalation/Interesting_Privileges.png?raw=true" alt="Interesting Privileges" width="70%">
<p style="text-align:center"><em>Table 10-1: Interesting Privileges</em></p>
</div>

Now that you know which privileges to look for, let's leverage Python to automatically retrieve the enabled privileges on the processes we're monitoring. We'll make use of the __win32security__, __win32api__, and __win32con__ modules. If you encounter a situation where you can't load these modules, try translating all of the following functions into native calls using the __ctypes__ library. This is possible, though it's a lot more work.

Add the following code to __process_monitor.py__ directly above the existing __log_to_file__ function:

In [ ]:
def get_process_privileges(pid):
    try:
        hproc = win32api.OpenProcess( #[1]
            win32con.PROCESS_QUERY_INFORMATION, False, pid
        )
        htok = win32security.OpenProcessToken(hproc, win32con.TOKEN_QUERY) #[2]
        privs = win32security.GetTokenInformation( #[3]
            htok, win32security.TokenPrivileges
        )
        privileges = ''
        for priv_id, flags in privs:
            if flag == (win32security.SE_PRIVILEGE_ENABLED | #[4]
                        win32security.SE_PRIVILEGE_ENABLED_BY_DEFAULT):
                privileges += f'{win32security.LookupPrivilegeName(None, priv_id)}|' #[5]
    except Exception:
        privileges = 'N/A'

    return privileges

We use the process ID to obtain a handle to the target process __[1]__. Next, we crack open the process token __[2]__ and request the token information for that process __[3]__ by sending the __win32security.TokenPrivileges__ structure. The function call returns a list of tuples, whre the first member of the tuple is the privilege and the second member describes whther the privilege is enabled or not. Because we're concenred with only the nabled ones, we first check for the enabled bits __[4]__ and then look up the human-readable name for that privilege __[5]__.

Next, modify the existing code to properly output and log this inforrmation. Change the line of code

```
privileges = 'N/A'
```

to the following:

```
privileges = get_process_privileges(pid)
```

Now that we've added the privilege-tracking code, let's rerun the __process_monitor.py__ script and check the output. You should see privilege information:

```
C:\Users\tim\work> python.exe process_monitor.py
"Calculator.exe",
2020062408445.120519-240 ,
C:\Program Files\WindowsApps\Microsoft.WindowsCalculator\Calculator.exe,
1204 ,
13116 ,
('DESKTOP-CC91N7I', 0, 'tim') ,
SeChangeNotifyPrivilege|

notepad ,
20200624084436.727998-240 ,
C:\Windows\system32\notepad.exe,
10720 ,
2732 ,
('DESKTOP-CC91N7I', 0, 'tim') ,
SeChangeNotifyPrivilege|SeImpersonatePrivilege|SeCreateGlobalPrivilege|
```

You can see that we've managed to log the enabled privileges for these processes. Now we could easily put some intelligence into the script to log only processes that run as an unprivileged user but have interesting privileges enabled. This use of process monitoring will let us find processes that rely on external files insecurely.